In [1]:
from src.board_games import noughts_and_crosses
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.metrics import accuracy_score

In [2]:
game = noughts_and_crosses.Game()

In [3]:
game.board

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [4]:
game.clear_board()

In [5]:
game._simulate_random_game(display=False, n=25000)

In [6]:
game.game_statistics()

--------------------------------
             
Total games:	25000
             
--------------------------------
             
Player 1 wins:	14446	(57.78%)
             
Player 2 wins:	7357	(29.43%)
             
Draws:		3197	(12.79%)
             
--------------------------------


In [7]:
board_states, winners = game._history_to_board_states()

In [8]:
def to_categorical(y, num_classes=None, dtype='float64'):
  y = np.array(y, dtype='int')
  input_shape = y.shape
  if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
    input_shape = tuple(input_shape[:-1])
  y = y.ravel()
  if not num_classes:
    num_classes = np.max(y) + 1
  n = y.shape[0]
  categorical = np.zeros((n, num_classes), dtype=dtype)
  categorical[np.arange(n), y] = 1
  output_shape = input_shape + (num_classes,)
  categorical = np.reshape(categorical, output_shape)
  return categorical

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

batch_size = 100
num_epochs = 100
num_classes = 3
learning_rate = 0.0001

outcome_states = to_categorical(winners, num_classes=num_classes)
game_results = noughts_and_crosses.GameResults(board_states, outcome_states)
entries = game_results.__len__()

train, test = random_split(game_results, [round(entries*0.8),round(entries*0.2)])

train_loader = DataLoader(train, batch_size=50, shuffle=True, num_workers=2)
test_loader = DataLoader(test, batch_size=50, shuffle=True, num_workers=2)

model = noughts_and_crosses.NeuralNetwork().to(device)

print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for batch, data in enumerate(train_loader):
        X, y = data['Board state'], data['Outcome state']
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch+1} loss: {loss:.4f}")

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=9, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=256, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=128, out_features=3, bias=True)
  )
)
Epoch 1 loss: 0.9466
Epoch 2 loss: 0.8919
Epoch 3 loss: 0.8249
Epoch 4 loss: 0.8984
Epoch 5 loss: 0.7816
Epoch 6 loss: 0.8789
Epoch 7 loss: 0.9325
Epoch 8 loss: 0.9028
Epoch 9 loss: 0.9450
Epoch 10 loss: 0.7820
Epoch 11 loss: 0.7343
Epoch 12 loss: 0.7847
Epoch 13 loss: 0.7763
Epoch 14 loss: 0.9051
Epoch 15 loss: 0.7367
Epoch 16 loss: 0.7728
Epoch 17 loss: 0.8224
Epoch 18 loss: 0.7721
Epoch 19 loss: 0.7162
Epoch 20 loss: 0.8556
Epoch 21 loss: 0.7322
Epoch 22 loss: 0.7

In [10]:
model.eval()
predictions = []
actual_winners = []

for batch, data in enumerate(test_loader):
    X, y = data['Board state'], data['Outcome state']
    X, y = X.to(device), y.to(device)
    
    pred = model(X)
    predictions.extend(pred.argmax(dim = -1).tolist())
    actual_winners.extend(y.argmax(dim= -1).tolist())

model_accuracy = accuracy_score(actual_winners, predictions)
print(f'The model was accurate {model_accuracy*100:.2f}% of the time.')

The model was accurate 67.09% of the time.


In [11]:
game.clear_board()

In [12]:
game.board

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [13]:
game._best_move(player=1, model=model)

array([1, 1], dtype=int64)